# Competitors research agent
Steps for competitors research:
1. Find list of competitors  
2. Go to their website  
3. Collect information including:  
	- standout features
	- product and pricing tiers
	- unique service proposition
	- marketing messages  
4. Analyze competitors  
	- identify common patterns  
	- spot potential gaps  
	- compare pricing strategies  
	- compare messaging themes

In [18]:
import aiohttp
import logging
import os
import json

import asyncio

from bs4 import BeautifulSoup
from datetime import datetime
from dotenv import load_dotenv
from google import genai
from typing import List, Dict, Any

In [19]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [20]:
class ResearchLogger:
    def __init__(self, research_id: str):
        self.research_id = research_id
        self.log_file = f"research_logs/{research_id}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
        self.json_log_file = f"research_logs/{research_id}_{datetime.now().strftime('%Y%m%d_%H%M%S')}_data.json"
        
        # Create logs directory if it doesn't exist
        os.makedirs("research_logs", exist_ok=True)
        
        # Set up file handler
        self.file_handler = logging.FileHandler(self.log_file)
        self.file_handler.setFormatter(
            logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        )
        
        # Add file handler to root logger
        logging.getLogger('').addHandler(self.file_handler)
        
        self.research_data = {
            "research_id": research_id,
            "timestamp": datetime.now().isoformat(),
            "steps": []
        }

    def log_step(self, agent_name: str, action: str, input_data: Any, output_data: Any):
        """Log a research step with both input and output data"""
        step_data = {
            "timestamp": datetime.now().isoformat(),
            "agent": agent_name,
            "action": action,
            "input": input_data,
            "output": output_data
        }
        
        self.research_data["steps"].append(step_data)
        
        # Log to file
        logger.info(f"Agent: {agent_name} | Action: {action}")
        logger.debug(f"Input: {json.dumps(input_data, indent=2)}")
        logger.debug(f"Output: {json.dumps(output_data, indent=2)}")
        
        # Save updated research data to JSON file
        self._save_research_data()

    def log_error(self, agent_name: str, action: str, error: Exception, context: Dict = None):
        """Log error information"""
        error_data = {
            "timestamp": datetime.now().isoformat(),
            "agent": agent_name,
            "action": action,
            "error": str(error),
            "error_type": type(error).__name__,
            "context": context
        }
        
        self.research_data["errors"] = self.research_data.get("errors", [])
        self.research_data["errors"].append(error_data)
        
        logger.error(f"Error in {agent_name} during {action}: {str(error)}")
        if context:
            logger.error(f"Context: {json.dumps(context, indent=2)}")
        
        self._save_research_data()

    def _save_research_data(self):
        """Save the complete research data to JSON file"""
        with open(self.json_log_file, 'w') as f:
            json.dump(self.research_data, f, indent=2)

    def get_research_summary(self) -> Dict:
        """Generate a summary of the research process"""
        return {
            "research_id": self.research_id,
            "total_steps": len(self.research_data["steps"]),
            "errors": len(self.research_data.get("errors", [])),
            "agents_involved": list(set(step["agent"] for step in self.research_data["steps"])),
            "duration": (datetime.now() - datetime.fromisoformat(self.research_data["timestamp"])).total_seconds()
        }

In [21]:
class RateLimiter:
    def __init__(self, calls_per_minute: int):
        self.calls_per_minute = calls_per_minute
        self.calls = []
        
    async def wait_if_needed(self):
        now = datetime.now()
        self.calls = [call for call in self.calls 
                     if (now - call).total_seconds() < 60]
        
        if len(self.calls) >= self.calls_per_minute:
            sleep_time = 60 - (now - self.calls[0]).total_seconds()
            await asyncio.sleep(sleep_time)
            
        self.calls.append(now)

In [22]:
class Config:
    def __init__(self):
        self.gemini_client = genai.Client(api_key=os.getenv('GOOGLE_API_KEY'))
        self.rate_limiter = RateLimiter(calls_per_minute=50)

In [23]:
class WebScraper:
    def __init__(self, config: Config, logger: ResearchLogger):
        self.config = config
        self.logger = logger
        self.subscription_key = os.getenv('BING_API_KEY')
        self.endpoint = 'https://api.bing.microsoft.com/v7.0/search'

    async def get_website_url(self, company_name: str) -> str:
        try:
            self.logger.log_step(
                agent_name="WebScraper",
                action="search_company_website",
                input_data={"company_name": company_name},
                output_data=None
            )

            headers = {'Ocp-Apim-Subscription-Key': self.subscription_key}
            params = {
                'q': f"{company_name} official website",
                'count': 1
            }
            
            async with aiohttp.ClientSession() as session:
                async with session.get(self.endpoint, headers=headers, params=params) as response:
                    data = await response.json()
                    website_url = data['webPages']['value'][0]['url']
                    
                    self.logger.log_step(
                        agent_name="WebScraper",
                        action="found_website_url",
                        input_data={"company_name": company_name},
                        output_data={"website_url": website_url}
                    )
                    
                    return website_url

        except Exception as e:
            self.logger.log_error(
                agent_name="WebScraper",
                action="get_website_url",
                error=e,
                context={"company_name": company_name}
            )
            raise

    async def extract_page_content(self, url: str) -> str:
        try:
            self.logger.log_step(
                agent_name="WebScraper",
                action="start_page_extraction",
                input_data={"url": url},
                output_data=None
            )

            async with aiohttp.ClientSession() as session:
                async with session.get(url) as response:
                    content = await response.text()
                    
                    # Use BeautifulSoup to parse and clean the content
                    soup = BeautifulSoup(content, 'html.parser')
                    
                    # Remove script and style elements
                    for script in soup(["script", "style"]):
                        script.decompose()
                    
                    # Get text and clean it
                    text = soup.get_text()
                    lines = (line.strip() for line in text.splitlines())
                    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
                    text = ' '.join(chunk for chunk in chunks if chunk)
                    
                    self.logger.log_step(
                        agent_name="WebScraper",
                        action="complete_page_extraction",
                        input_data={"url": url},
                        output_data={"content_length": len(text)}
                    )
                    
                    return text

        except Exception as e:
            self.logger.log_error(
                agent_name="WebScraper",
                action="extract_page_content",
                error=e,
                context={"url": url}
            )
            raise

    async def analyze_company(self, company_name: str) -> Dict:
        try:
            self.logger.log_step(
                agent_name="WebScraper",
                action="start_company_analysis",
                input_data={"company_name": company_name},
                output_data=None
            )

            website_url = await self.get_website_url(company_name)
            
            # Define common page paths to check
            pages_to_analyze = [
                {"path": "", "type": "home"},
                {"path": "pricing", "type": "pricing"},
                {"path": "products", "type": "products"},
                {"path": "about", "type": "about"},
                {"path": "features", "type": "features"},
                {"path": "solutions", "type": "solutions"}
            ]
            
            company_data = {
                "name": company_name,
                "website": website_url,
                "pages": {}
            }

            for page in pages_to_analyze:
                try:
                    page_url = f"{website_url.rstrip('/')}/{page['path']}"
                    
                    self.logger.log_step(
                        agent_name="WebScraper",
                        action="analyze_page",
                        input_data={
                            "company_name": company_name,
                            "page_type": page["type"],
                            "url": page_url
                        },
                        output_data=None
                    )
                    
                    content = await self.extract_page_content(page_url)
                    company_data["pages"][page["type"]] = {
                        "url": page_url,
                        "content": content
                    }
                    
                except Exception as e:
                    self.logger.log_error(
                        agent_name="WebScraper",
                        action="analyze_page",
                        error=e,
                        context={
                            "company_name": company_name,
                            "page_type": page["type"],
                            "url": page_url
                        }
                    )
                    continue

            self.logger.log_step(
                agent_name="WebScraper",
                action="complete_company_analysis",
                input_data={"company_name": company_name},
                output_data={
                    "website": website_url,
                    "pages_analyzed": list(company_data["pages"].keys())
                }
            )

            return company_data

        except Exception as e:
            self.logger.log_error(
                agent_name="WebScraper",
                action="analyze_company",
                error=e,
                context={"company_name": company_name}
            )
            raise

    async def analyze_competitors(self, competitors_data: Dict) -> Dict:
        print(type(competitors_data))
        if isinstance(competitors_data, str):
            competitors_data = json.loads(competitors_data)
            
        try:
            self.logger.log_step(
                agent_name="WebScraper",
                action="start_competitors_analysis",
                input_data={"competitors": [comp["name"] for comp in competitors_data["competitors"]]},
                output_data=None
            )

            results = {}
            for competitor in competitors_data["competitors"]:
                try:
                    results[competitor["name"]] = await self.analyze_company(competitor["name"])
                except Exception as e:
                    self.logger.log_error(
                        agent_name="WebScraper",
                        action="analyze_competitor",
                        error=e,
                        context={"competitor": competitor}
                    )
                    continue

            self.logger.log_step(
                agent_name="WebScraper",
                action="complete_competitors_analysis",
                input_data={"competitors": [comp["name"] for comp in competitors_data["competitors"]]},
                output_data={
                    "competitors_analyzed": list(results.keys()),
                    "total_competitors": len(competitors_data["competitors"]),
                    "successful_analyses": len(results)
                }
            )

            return results

        except Exception as e:
            self.logger.log_error(
                agent_name="WebScraper",
                action="analyze_competitors",
                error=e,
                context={"competitors_data": competitors_data}
            )
            raise

    async def extract_structured_data(self, content: str, data_type: str) -> Dict:
        """Extract specific types of data from page content"""
        try:
            self.logger.log_step(
                agent_name="WebScraper",
                action="start_structured_data_extraction",
                input_data={"data_type": data_type},
                output_data=None
            )

            # Use Gemini to extract structured data
            prompt = f"""
            Extract the following type of information: {data_type}
            From the following content:
            {content[:1000]}  # Limit content length for API
            
            Return the information in JSON format.
            """
            
            response = self.config.gemini_client.models.generate_content(
                model="gemini-2.0-flash-exp",
                contents=prompt
            )
            
            structured_data = json.loads(response.text)
            
            self.logger.log_step(
                agent_name="WebScraper",
                action="complete_structured_data_extraction",
                input_data={"data_type": data_type},
                output_data=structured_data
            )
            
            return structured_data

        except Exception as e:
            self.logger.log_error(
                agent_name="WebScraper",
                action="extract_structured_data",
                error=e,
                context={"data_type": data_type}
            )
            raise

In [24]:
class BaseAgent:
    def __init__(self, config: Config, logger: ResearchLogger):
        self.config = config
        self.logger = logger
        self.system_prompt = ""
        self.role = ""
        self.goal = ""
        self.backstory = ""

    async def execute(self, input_data: Any) -> Any:
        await self.config.rate_limiter.wait_if_needed()
        
        try:
            # Log the start of execution
            self.logger.log_step(
                agent_name=self.__class__.__name__,
                action="start_execution",
                input_data=input_data,
                output_data=None
            )
            
            prompt = f"""
            Role: {self.role}
            Goal: {self.goal}
            Backstory: {self.backstory}
            System Instructions: {self.system_prompt}
            
            Input Data:
            {json.dumps(input_data, indent=2)}
            
            Please provide your analysis based on the above information.
            """
            
            response = self.config.gemini_client.models.generate_content(
                model="gemini-2.0-flash-exp",
                contents=prompt
            )
            
            result = json.loads(response.text.replace('```json', '').replace('```', '').strip())
            
            # Log the successful execution
            self.logger.log_step(
                agent_name=self.__class__.__name__,
                action="complete_execution",
                input_data=input_data,
                output_data=result
            )
            
            return result
            
        except Exception as e:
            # Log the error
            self.logger.log_error(
                agent_name=self.__class__.__name__,
                action="execute",
                error=e,
                context={"input_data": input_data}
            )
            raise

In [25]:
class MarketIntelligenceScout(BaseAgent):
    def __init__(self, config: Config, logger: ResearchLogger):
        super().__init__(config, logger)
        self.role = "Expert market researcher specializing in competitor identification"
        self.goal = "Identify and categorize the most relevant competitors"
        self.backstory = "Former market research director with 15 years of experience"
        self.system_prompt = """
        1. Search for top competitors in the given market
        2. Return results in JSON format with the following structure:
        {
            "competitors": [
                {
                    "name": "",
                    "website": "",
                    "industry": "",
                    "threat_level": ""
                }
            ]
        }
        """

In [26]:
class DigitalProductAnalyst(BaseAgent):
    def __init__(self, config: Config, logger: ResearchLogger):
        super().__init__(config, logger)
        self.role = "Product analysis specialist"
        self.goal = "Analyze product features and capabilities"
        self.backstory = "Previously a product manager at major tech companies"
        self.system_prompt = """
        Analyze each competitor's product features and capabilities.
        Return results in JSON format:
        {
            "competitor_name": {
                "key_features": [],
                "unique_capabilities": [],
                "user_experience": "",
                "product_maturity": ""
            }
        }
        """

In [27]:
class MarketingMessageDecoder(BaseAgent):
    def __init__(self, config: Config, logger: ResearchLogger):
        super().__init__(config, logger)
        self.role = "Marketing communications analyst"
        self.goal = "Decode and analyze competitors' marketing strategies"
        self.backstory = "Former copywriter turned marketing strategist"
        self.system_prompt = """
        Analyze marketing messages and positioning.
        Return results in JSON format:
        {
            "competitor_name": {
                "value_propositions": [],
                "messaging_tone": "",
                "target_audience": "",
                "unique_selling_points": []
            }
        }
        """

In [28]:
class TechnicalFeatureComparator(BaseAgent):
    def __init__(self, config: Config, logger: ResearchLogger):
        super().__init__(config, logger)
        self.role = "Technical analyst specializing in feature comparison"
        self.goal = "Provide detailed technical comparison of competitor products"
        self.backstory = "Senior solutions architect with cross-industry experience"
        self.system_prompt = """
        Compare technical features across competitors.
        Return results in JSON format:
        {
            "competitor_name": {
                "tech_stack": [],
                "api_capabilities": [],
                "scalability_features": [],
                "technical_advantages": [],
                "technical_limitations": []
            }
        }
        """

In [29]:
class PricingStrategySpecialist(BaseAgent):
    def __init__(self, config: Config, logger: ResearchLogger):
        super().__init__(config, logger)
        self.role = "Pricing analysis expert"
        self.goal = "Analyze and compare pricing models and strategies"
        self.backstory = "Former pricing consultant in SaaS industry"
        self.system_prompt = """
        Analyze pricing strategies and models.
        Return results in JSON format:
        {
            "competitor_name": {
                "pricing_tiers": [],
                "pricing_model": "",
                "discount_strategies": [],
                "pricing_positioning": ""
            }
        }
        """

In [30]:
class CompetitiveStrategyAnalyst(BaseAgent):
    def __init__(self, config: Config, logger: ResearchLogger):
        super().__init__(config, logger)
        self.role = "Strategic analyst specializing in competitive analysis"
        self.goal = "Synthesize competitive intelligence into strategic insights"
        self.backstory = "Strategy consultant from major consulting firms"
        self.system_prompt = """
        Synthesize all competitive data into strategic insights.
        Return results in JSON format:
        {
            "market_patterns": [],
            "competitive_advantages": {},
            "market_gaps": [],
            "strategic_recommendations": [],
            "threat_assessment": {}
        }
        """

In [31]:
class CompetitiveIntelligenceReportSpecialist(BaseAgent):
    def __init__(self, config: Config, logger: ResearchLogger):
        super().__init__(config, logger)
        self.role = "Report creation specialist"
        self.goal = "Create clear, actionable reports from competitive analysis"
        self.backstory = "Communications expert in data visualization"
        self.system_prompt = """
        Create a comprehensive report from all analyses.
        Return results in JSON format:
        {
            "executive_summary": "",
            "key_findings": [],
            "detailed_analysis": {},
            "recommendations": [],
            "market_overview": "",
            "appendix": {}
        }
        """

In [32]:
class CompetitiveAnalysisWorkflow:
    def __init__(self, config: Config, research_id: str):
        self.config = config
        self.logger = ResearchLogger(research_id)

        self.market_scout = MarketIntelligenceScout(config, self.logger)
        self.product_analyst = DigitalProductAnalyst(config, self.logger)
        self.marketing_decoder = MarketingMessageDecoder(config, self.logger)
        self.technical_comparator = TechnicalFeatureComparator(config, self.logger)
        self.pricing_specialist = PricingStrategySpecialist(config, self.logger)
        self.competitive_analyst = CompetitiveStrategyAnalyst(config, self.logger)
        self.report_specialist = CompetitiveIntelligenceReportSpecialist(config, self.logger)
        self.web_scraper = WebScraper(config, self.logger)

    async def run_parallel_analysis(self, competitors_data: Dict) -> Dict[str, Any]:
        """Run parallel analysis tasks"""
        tasks = [
            self.product_analyst.execute(competitors_data),
            self.marketing_decoder.execute(competitors_data),
            self.technical_comparator.execute(competitors_data),
            self.pricing_specialist.execute(competitors_data)
        ]
        
        results = await asyncio.gather(*tasks, return_exceptions=True)
        
        return {
            "product_analysis": results[0],
            "marketing_analysis": results[1],
            "technical_analysis": results[2],
            "pricing_analysis": results[3]
        }

    async def execute_workflow(self, industry: str, target_company: str = None) -> Dict:
        try:
            self.logger.log_step(
                agent_name="Workflow",
                action="start_workflow",
                input_data={"industry": industry, "target_company": target_company},
                output_data=None
            )
            
            # Step 1: Retrieve target company information (if necessar;)
            target_company_data = None
            if target_company:
                target_company_data = await self.web_scraper.analyze_company(target_company)

            # Step 2: Identify competitors
            competitors_data = await self.market_scout.execute({
                "industry": industry,
                "target_company": target_company,
                "target_company_data": target_company_data
            })

            # Step 3: Gather website information for all competitors
            website_data = await self.web_scraper.analyze_competitors(competitors_data)

            # Step 4: Run parallel analysis with website data
            parallel_results = await self.run_parallel_analysis({
                "competitors_data": competitors_data,
                "website_data": website_data,
                "target_company_data": target_company_data
            })

            # Step 3: Strategic analysis
            strategic_analysis = await self.competitive_analyst.execute({
                "competitors_data": competitors_data,
                "parallel_results": parallel_results
            })

            # Step 4: Generate report
            final_report = await self.report_specialist.execute({
                "strategic_analysis": strategic_analysis,
                "raw_data": {
                    "competitors": competitors_data,
                    "analysis": parallel_results
                }
            })

            self.logger.log_step(
                agent_name="Workflow",
                action="complete_workflow",
                input_data={"industry": industry, "target_company": target_company},
                output_data=None
            )

            return final_report

        except Exception as e:
            self.logger.log_error(
                agent_name="Workflow",
                action="execute_workflow",
                error=e,
                context={
                    "industry": industry,
                    "target_company": target_company
                }
            )
            raise

In [33]:
async def test_workflow():
    config = Config()
    research_id = f"research_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    
    workflow = CompetitiveAnalysisWorkflow(config, research_id)
    
    industry = "AI development platforms"
    target_company = "OpenAI"  # Optional, can be None
    
    result = await workflow.execute_workflow(
        industry=industry,
        target_company=target_company
    )
    
    # Print research summary
    print("\nResearch Summary:")
    print(json.dumps(workflow.logger.get_research_summary(), indent=2))
    
    print("\nFinal Result:")
    print(json.dumps(result, indent=2))
    
    return result

In [17]:
result = await test_workflow()

INFO:__main__:Agent: Workflow | Action: start_workflow
INFO:__main__:Agent: WebScraper | Action: start_company_analysis
INFO:__main__:Agent: WebScraper | Action: search_company_website
INFO:__main__:Agent: WebScraper | Action: found_website_url
INFO:__main__:Agent: WebScraper | Action: analyze_page
INFO:__main__:Agent: WebScraper | Action: start_page_extraction
INFO:__main__:Agent: WebScraper | Action: complete_page_extraction
INFO:__main__:Agent: WebScraper | Action: analyze_page
INFO:__main__:Agent: WebScraper | Action: start_page_extraction
INFO:__main__:Agent: WebScraper | Action: complete_page_extraction
INFO:__main__:Agent: WebScraper | Action: analyze_page
INFO:__main__:Agent: WebScraper | Action: start_page_extraction
INFO:__main__:Agent: WebScraper | Action: complete_page_extraction
INFO:__main__:Agent: WebScraper | Action: analyze_page
INFO:__main__:Agent: WebScraper | Action: start_page_extraction
INFO:__main__:Agent: WebScraper | Action: complete_page_extraction
INFO:__main

> /var/folders/mk/spx5m7bs31zcl288wyr4sdfm0000gn/T/ipykernel_33881/3431605583.py(39)execute()
     37             )
     38 
---> 39             import pdb; pdb.set_trace()
     40             result = json.loads(response.text)
     41 

*** AttributeError: 'GenerateContentResponse' object has no attribute 'txt'. Did you mean: 'text'?
'```json\n{\n    "competitors": [\n        {\n            "name": "Google AI",\n            "website": "https://ai.google/",\n            "industry": "AI development platforms",\n             "threat_level": "High"\n        },\n        {\n            "name": "Microsoft Azure AI",\n            "website": "https://azure.microsoft.com/en-us/products/ai/",\n            "industry": "AI development platforms",\n             "threat_level": "High"\n        },\n        {\n            "name": "Amazon SageMaker",\n             "website": "https://aws.amazon.com/sagemaker/",\n            "industry": "AI development platforms",\n             "threat_level": "High"\n 

In [18]:
def save_json_to_markdown(json_str, output_file="output.md"):
    """
    Converts a JSON string to a Markdown formatted string and saves it to a file.
    Handles incomplete JSON strings due to potential truncation.

    Args:
        json_str: A string containing potentially incomplete JSON data.
        output_file: The name of the file to save the markdown output to. Defaults to "output.md".

    Returns:
        A string containing the Markdown representation of the JSON data, or an error message.
        Also saves the markdown output to the specified file if successful.
    """
    # Remove the ```json\n and \n from the string and strip whitespace
    json_str = json_str.replace("```json\n", "").strip()

    # Check if the string is valid JSON and attempt to repair minimal issues
    try:
        data = json.loads(json_str)
    except json.JSONDecodeError:
        # Repair: Add closing braces or truncate to last valid index
        braces_balance = json_str.count('{') - json_str.count('}')
        brackets_balance = json_str.count('[') - json_str.count(']')
        if braces_balance > 0:
            json_str += '}' * braces_balance
        elif brackets_balance > 0:
            json_str += ']' * brackets_balance
        else:
            last_valid_index = max(json_str.rfind('}'), json_str.rfind(']'))
            json_str = json_str[:last_valid_index + 1]
        
        # Re-validate JSON
        try:
            data = json.loads(json_str)
        except json.JSONDecodeError as e:
            return f"Error: Invalid JSON string after repair: {e}"

    # Convert JSON to Markdown
    def json_to_markdown(data, level=0):
        indent = "  " * level
        lines = []
        if isinstance(data, dict):
            for key, value in data.items():
                lines.append(f"{indent}**{key.replace('_', ' ').title()}:**")
                lines.extend(json_to_markdown(value, level + 1))
        elif isinstance(data, list):
            for item in data:
                lines.extend(json_to_markdown(item, level + 1))
        else:
            lines.append(f"{indent}- {data}")
        return lines

    markdown_output = "\n".join(json_to_markdown(data))

    # Save to file
    try:
        with open(output_file, "w", encoding="utf-8") as f:
            f.write(markdown_output)
    except Exception as e:
        return f"Error: Could not save to file: {e}"

    return markdown_output


In [19]:
save_json_to_markdown(result, report_name)

"**Executive Summary:**\n  - The AI/ML market is characterized by a mix of cloud-based platforms, open-source frameworks, and specialized solutions. Key players include Google, Amazon, Microsoft, and IBM, each offering comprehensive AI/ML services with varying strengths. Open-source options like TensorFlow and PyTorch are critical, while AutoML solutions from DataRobot and H2O.ai cater to different user needs. Significant market gaps exist around MLOps simplification, XAI, model monitoring, cost transparency, and hardware accelerator utilization. This report provides a detailed analysis of the competitive landscape, highlighting key findings, recommendations, and market opportunities.\n**Key Findings:**\n    - Cloud-based platforms from Google, Amazon, and Microsoft dominate the market, providing extensive AI/ML services.\n    - Open-source frameworks like TensorFlow and PyTorch are fundamental for much of AI/ML development, but are not fully encompassing solutions.\n    - Automated Ma